- # Business Problem Description
- We are looking at a dataset of Bank
- They mesaured some things about customers
- They are noticing some unusual churn rate (customers leaving the company) at an unusual high rate
- They want to access what the problem is...
- They took 10000 observation from customers and waited for 6 months to identify which of these customers left and which did not
- Left the Bank (1), Haven't left (0)
- Notify the bank which customers are at a high risk of leaving

# Import Library

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
tf.__version__

'2.13.0'

# Part 1 - Data Preprocessing

### Import Dataset

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:,3:-1].values
Y = dataset.iloc[:,-1].values

## Encoding Categorical Data

### Label Encoding the  "Gender" Column

In [4]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
X[:,2] = le.fit_transform(X[:,2])

### One Hot Encoding the "Geography" Column

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

## Splitting into train and test set

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 1)

## Feature Scaling

Fundamental and Essential for Deep Learning

In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Part 2 - Building the ANN

## Initializing the ANN

In [8]:
ann = tf.keras.models.Sequential()

## Adding the input and the first hidden layer

no rule of thumbs and must try and experiment for good result 

In [9]:
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

## Adding the second hidden layer

In [10]:
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

## Adding the output layer

- If the output is going to be single then set the unit to be binary. If the dependent variable is say a,b,c then we need 3 dimensions (output neurons) then OneHotEncoding 
- if its non binary then the activation must be softmax

In [11]:
ann.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

# Part 3 - Training the ANN

## Compiling the ANN

- adam performs 'Gradient Descent'
- when performing binary outcome the loss function must always be binary_crossentropy for non binary (categorical_crossentropy)

In [12]:
ann.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

## Training the ANN on the Training Set

- default value 32

In [13]:
ann.fit(X_train,y_train,batch_size= 32,epochs=100)

Epoch 1/100
250/250 [==============================] - 2s 3ms/step - loss: 0.5498 - accuracy: 0.7780
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4778 - accuracy: 0.7976
Epoch 3/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4520 - accuracy: 0.8008
Epoch 4/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4351 - accuracy: 0.8064
Epoch 5/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4237 - accuracy: 0.8139
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4127 - accuracy: 0.8227
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3992 - accuracy: 0.8330
Epoch 8/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3870 - accuracy: 0.8399
Epoch 9/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3769 - accuracy: 0.8440
Epoch 10/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3689 - accura

# Part 4 - Making the prediction and evaluating the model

## Predicting the result of single observation

**Homework**

Use our ANN model to predict if the customer with the following informations will leave the bank: 

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card ? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer ?

**Solution**

In [21]:
print(ann.predict(sc.transform([[1,1,0,600,1,40,3,60000,2,1,1,50000]])) > 0.5 )

1/1 [==============================] - 0s 48ms/step
[[False]]


## Predicting the Test set results

In [28]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

63/63 [==============================] - 0s 4ms/step
[[0 0]
 [0 0]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


## Making the Confusion Matrix

In [29]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1543   42]
 [ 229  186]]


0.8645